# Testing framework "pipeline"

_Alex Malz (NYU)_

In [ ]:
import numpy as np
import scipy.stats as sps
from scipy.stats import rv_discrete as rvd
import matplotlib.pyplot as plt
%matplotlib inline

import proclam
from proclam import *

## 1. True classes

Import the `proclam.Simulator` superclass and the simulator subclass you want to test.  In this notebook, I'm going to use a uniform distribution of true classes.

In [ ]:
from proclam.simulators import simulator
from proclam.simulators import scipydist

Create an instantiation of the simulator for the true dataset class distribution scheme.  If you use the base superclass instead of a subclass, the default scheme will be to assign all objects the same class.

Then, simulate a truth catalog.  In this case, there are 3 true classes and 100 objects in the catalog.  The output will be a `numpy.ndarray` with 100 entries, each of which is the index of the class for that catalog member.

In [ ]:
M = 3

In [ ]:
distA = sps.randint(0, M)
simA = proclam.simulators.scipydist.SciPyDist(scheme=distA)
truthA = simA.simulate(100, M)

In [ ]:
distB = sps.poisson(0.5)
simB = proclam.simulators.scipydist.SciPyDist(scheme=distB)
truthB = simB.simulate(100, M)

In [ ]:
distC = sps.binom(M-1, 0.5)
simC = proclam.simulators.scipydist.SciPyDist(scheme=distC)
truthC = simC.simulate(100, M)

In [ ]:
plt.hist(truthA, alpha=0.5, label='randint')
# plt.hist(truthB, alpha=0.5, label='poisson')
plt.hist(truthC, alpha=0.5, label='binom')
plt.legend()

## 2. Mock classifications

Import the `proclam.Classifier` superclass and the classifier subclass you want to test.  In this notebook, I'm going to use a very stupid classifier that takes a random guess.

In [ ]:
from proclam.classifiers import classifier
from proclam.classifiers import guess

Create an instantiation of the classifier of a particular scheme.  If you use the base superclass instead of a subclass, the default classification scheme will return the true classes.

Then, "classify" the "data."  By default, classification results will also include an extra column for "other" classes beyond the number in the training set, but in this example let's assume it knows of the 3 classes in the training set without leaving room for additional classes.  The output will be a `numpy.ndarray` with 100 rows and column entries representing each catalog member's posterior probability for being of each class.

In [ ]:
guesser = proclam.classifiers.guess.Guess()
prediction = guesser.classify(3, truthA, other=False)
# print(prediction)

## 3. Metric values

Import the `proclam.Metric` superclass and the metric subclass you want to test.  I haven't implemented any `Metric` subclasses yet, so we'll have to make do with the base superclass.

In [ ]:
from proclam.metrics import metric

Create an instantiation of the metric.  The default behavior of the base superclass is to take the maximum probability class as the true class and calculate an accuracy, equally weighting over all catalog members.

Then, calculate the metric value.

In [ ]:
C = proclam.metrics.metric.Metric()
performance = C.evaluate(prediction, truth)
# print(performance)

# Next steps

Once there are more simulators, classifiers, and metrics, we'll loop over tests and plot comparisons.  Stay tuned for more!